In [1]:
!nvidia-smi

Sun Dec 28 12:08:32 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.172.08             Driver Version: 570.172.08     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   37C    P0             26W /  250W |       0MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pwd

/kaggle/working


In [ ]:
!git clone https://github.com/KapKr21/fl-project.git

In [3]:
!ls
%cd fl-project

fl-project  model_56000.zip  model_60000.zip
/kaggle/working/fl-project


In [4]:
!ls
!git status
!git branch

examples  LICENSE.md  loralib  README.md  SECURITY.md  setup.py
On branch lora-fine-tuning
Your branch is ahead of 'origin/lora-fine-tuning' by 1 commit.
  (use "git push" to publish your local commits)

Changes not staged for commit:
  (use "git add/rm <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   examples/NLG/eval/e2e/pycocoevalcap/meteor/data/paraphrase-en.gz
	deleted:    examples/NLU/docs/source/contributing.md
	deleted:    examples/NLU/docs/source/examples.md
	deleted:    examples/NLU/docs/source/notebooks.md
	modified:   examples/NLU/examples/benchmarking/run_benchmark.py
	modified:   examples/NLU/examples/benchmarking/run_benchmark_tf.py
	modified:   examples/NLU/examples/language-modeling/run_clm.py
	modified:   examples/NLU/examples/language-modeling/run_mlm.py
	modified:   examples/NLU/examples/language-modeling/run_mlm_flax.py
	modified:   examples/NLU/examples/language-modeling/run_plm.py
	m

In [5]:
!pip -q install -U pip
!pip -q install spacy tqdm tensorboard progress loralib
!pip -q install -U "transformers>=4.41,<6"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 29.7 MB/s eta 0:00:00a 0:00:01


In [2]:
%cd examples/NLG
!bash download_pretrained_checkpoints.sh

[Errno 2] No such file or directory: 'examples/NLG'
/kaggle/working
bash: download_pretrained_checkpoints.sh: No such file or directory


In [7]:
!pip install "numpy<2"

In [ ]:
!bash create_datasets.sh

In [ ]:
%cd eval
!bash download_evalscript.sh
%cd ..


In [8]:
%env PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

env: PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True


In [ ]:
!torchrun --nproc_per_node=1 src/gpt2_ft.py \
  --train_data ./data/e2e/train.jsonl \
  --valid_data ./data/e2e/valid.jsonl \
  --train_batch_size 2 \
  --grad_acc 4 \
  --valid_batch_size 1 \
  --seq_len 512 \
  --model_card gpt2.md \
  --init_checkpoint ./pretrained_checkpoints/gpt2-medium-pytorch_model.bin \
  --platform local \
  --clip 0.0 \
  --lr 0.0002 \
  --weight_decay 0.01 \
  --correct_bias \
  --adam_beta2 0.999 \
  --scheduler linear \
  --warmup_step 2000 \
  --max_epoch 2 \
  --save_interval 4000 \
  --lora_dim 4 \
  --lora_alpha 32 \
  --lora_dropout 0.1 \
  --label_smooth 0.1 \
  --work_dir ./trained_models/GPT2_M/e2e \
  --random_seed 110

In [ ]:
!git status

In [4]:
!ls
%cd examples/NLG
!ls

examples  LICENSE.md  loralib  README.md  SECURITY.md  setup.py
/kaggle/working/fl-project/examples/NLG
CODE_OF_CONDUCT.md		    eval		    SECURITY.md
create_datasets.sh		    figures		    src
data				    LICENSE		    state.db
download_pretrained_checkpoints.sh  pretrained_checkpoints  trained_models
e2e_pred.txt			    README.md		    vocab
e2e_ref.txt			    requirement.txt


In [ ]:
!torchrun --nproc_per_node=1 src/gpt2_beam.py \
  --data ./data/e2e/test.jsonl \
  --batch_size 1 \
  --seq_len 512 \
  --eval_len 64 \
  --model_card gpt2.md \
  --init_checkpoint ./trained_models/GPT2_M/e2e/model.42062.pt \
  --platform local \
  --lora_dim 4 \
  --lora_alpha 32 \
  --beam 10 \
  --length_penalty 0.8 \
  --no_repeat_ngram_size 4 \
  --repetition_penalty 1.0 \
  --eos_token_id 628 \
  --work_dir ./trained_models/GPT2_M/e2e \
  --output_file predict.42062.b10p08r4.jsonl

In [11]:
!wc -l trained_models/GPT2_M/e2e/predict.42062.b10p08r4.jsonl

4693 trained_models/GPT2_M/e2e/predict.42062.b10p08r4.jsonl


In [12]:
!python src/gpt2_decode.py \
  --vocab ./vocab \
  --sample_file ./trained_models/GPT2_M/e2e/predict.42062.b10p08r4.jsonl \
  --input_file ./data/e2e/test_formatted.jsonl \
  --output_ref_file e2e_ref.txt \
  --output_pred_file e2e_pred.txt

/kaggle/working/fl-project/examples/NLG/src/gpt2_decode.py:50: SyntaxWarning: invalid escape sequence '\W'
  sent = ' '.join(re.split('(\W)', sent))
unique refer dict 630


In [13]:
!wc -l e2e_ref.txt e2e_pred.txt

  5323 e2e_ref.txt
   630 e2e_pred.txt
  5953 total


In [14]:
!python eval/e2e/measure_scores.py e2e_ref.txt e2e_pred.txt -p

/kaggle/working/fl-project/examples/NLG/eval/e2e/measure_scores.py:208: SyntaxWarning: invalid escape sequence '\.'
  elif re.search('\.[ct]sv$', sys_file, re.I):
/kaggle/working/fl-project/examples/NLG/eval/e2e/measure_scores.py:216: SyntaxWarning: invalid escape sequence '\.'
  if re.search('\.[ct]sv$', ref_file, re.I):
Running MS-COCO evaluator...
creating index...
index created!
Loading and preparing results...     
DONE (t=0.00s)
creating index...
index created!
tokenization...
PTBTokenizer tokenized 132573 tokens at 368342.79 tokens per second.
PTBTokenizer tokenized 18540 tokens at 122113.38 tokens per second.
setting up scorers...
computing METEOR score...
METEOR: 0.455
computing Rouge score...
ROUGE_L: 0.694
computing CIDEr score...
CIDEr: 2.406
Running Py-MTEval metrics...
SCORES:
BLEU: 0.6759
NIST: 8.6072
METEOR: 0.4546
ROUGE_L: 0.6938
CIDEr: 2.4059



In [ ]:
#Retraining with more steps i.e. 100000+

In [ ]:
!torchrun --nproc_per_node=1 src/gpt2_ft.py \
  --train_data ./data/e2e/train.jsonl \
  --valid_data ./data/e2e/valid.jsonl \
  --train_batch_size 2 \
  --grad_acc 4 \
  --valid_batch_size 1 \
  --seq_len 512 \
  --model_card gpt2.md \
  --init_checkpoint ./pretrained_checkpoints/gpt2-medium-pytorch_model.bin \
  --platform local \
  --clip 0.0 \
  --lr 0.0002 \
  --weight_decay 0.01 \
  --correct_bias \
  --adam_beta2 0.999 \
  --scheduler linear \
  --warmup_step 2000 \
  --max_epoch 5 \
  --save_interval 4000 \
  --lora_dim 4 \
  --lora_alpha 32 \
  --lora_dropout 0.1 \
  --label_smooth 0.1 \
  --work_dir ./trained_models/GPT2_M/e2e_seed110_full \
  --random_seed 110

myrank: 0 local_rank: 0 device_count: 1 world_size: 1
        - platform : local
        - local_rank : 0
        - rank : 0
        - device : cuda:0
        - world_size : 1
        - random_seed : 110
        - lr : 0.0002
        - weight_decay : 0.01
        - correct_bias : True
        - adam_epislon : 1e-06
        - no_decay_bias : False
        - adam_beta1 : 0.9
        - adam_beta2 : 0.999
        - scheduler : linear
        - max_step : None
        - max_epoch : 5
        - warmup_step : 2000
        - i_steps : 0
        - i_lrs : 0.00025
        - train_data : ./data/e2e/train.jsonl
        - valid_data : ./data/e2e/valid.jsonl
        - train_batch_size : 2
        - valid_batch_size : 1
        - grad_acc : 4
        - clip : 0.0
        - seq_len : 512
        - model_card : gpt2.md
        - init_checkpoint : ./pretrained_checkpoints/gpt2-medium-pytorch_model.bin
        - fp16 : False
        - log_interval : 100
        - eval_interval : 2000
        - save_inter

In [5]:
!ls -lh trained_models/GPT2_M/e2e_seed110_full | tail -n 30
!ls trained_models/GPT2_M/e2e_seed110_full | grep -E "model\.[0-9]+\.pt" | sort -V | tail -n 5


total 2.9G
-rw-r--r-- 1 root root 1.6M Dec 29 00:34 model.12000.pt
-rw-r--r-- 1 root root 1.6M Dec 29 00:34 model.16000.pt
-rw-r--r-- 1 root root 1.6M Dec 29 00:34 model.20000.pt
-rw-r--r-- 1 root root 1.5G Dec 29 00:34 model.21031.pt
-rw-r--r-- 1 root root 1.6M Dec 29 00:34 model.24000.pt
-rw-r--r-- 1 root root 1.6M Dec 29 00:34 model.28000.pt
-rw-r--r-- 1 root root 1.6M Dec 29 00:34 model.32000.pt
-rw-r--r-- 1 root root 1.6M Dec 29 00:34 model.36000.pt
-rw-r--r-- 1 root root 1.6M Dec 29 00:34 model.40000.pt
-rw-r--r-- 1 root root 1.6M Dec 29 00:34 model.4000.pt
-rw-r--r-- 1 root root 1.5G Dec 29 00:34 model.42062.pt
-rw-r--r-- 1 root root 1.6M Dec 29 00:34 model.44000.pt
-rw-r--r-- 1 root root 1.6M Dec 29 00:34 model.48000.pt
-rw-r--r-- 1 root root 1.6M Dec 29 00:34 model.52000.pt
-rw-r--r-- 1 root root 1.6M Dec 29 00:34 model.56000.pt
-rw-r--r-- 1 root root 1.6M Dec 29 00:34 model.60000.pt
-rw-r--r-- 1 root root 1.6M Dec 29 00:34 model.8000.pt
model.44000.pt
model.48000.pt
model.520

In [2]:
!ls
%cd fl-project/examples/NLG
!ls

fl-project  model_60000.zip
/kaggle/working/fl-project/examples/NLG
CODE_OF_CONDUCT.md		    eval		    SECURITY.md
create_datasets.sh		    figures		    src
data				    LICENSE		    state.db
download_pretrained_checkpoints.sh  pretrained_checkpoints  trained_models
e2e_pred.txt			    README.md		    vocab
e2e_ref.txt			    requirement.txt


In [3]:
!zip -j /kaggle/working/model_56000.zip trained_models/GPT2_M/e2e_seed110_full/model.56000.pt

  adding: model.56000.pt (deflated 8%)


In [4]:
!wc -l data/e2e/test.jsonl

4693 data/e2e/test.jsonl
